In [6]:
import os
from PIL import Image

base_dir = "C:/Users/stszy/jupyterNoteBookNotes/cellDetector/model/LearnSet145"
input_img_dir = os.path.join(base_dir, "images", "train")
input_lbl_dir = os.path.join(base_dir, "labels", "train")

output_img_dir = os.path.join(base_dir, "images_sliced", "train")
output_lbl_dir = os.path.join(base_dir, "labels_sliced", "train")

os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_lbl_dir, exist_ok=True)

image_ext = [".jpg", ".png", ".jpeg", ".tif", ".tiff"]

tile_size = 1024  

def convert_bbox(bbox, orig_w, orig_h):
    cx, cy, w, h = bbox
    return [
        (cx - w / 2) * orig_w,
        (cy - h / 2) * orig_h,
        (cx + w / 2) * orig_w,
        (cy + h / 2) * orig_h
    ]


def crop_images_and_labels():
    for fname in os.listdir(input_img_dir):
        name, ext = os.path.splitext(fname)
        if ext.lower() not in image_ext:
            continue

        img_path = os.path.join(input_img_dir, fname)
        label_path = os.path.join(input_lbl_dir, name + ".txt")

        img = Image.open(img_path)
        width, height = img.size

        if width % tile_size != 0 or height % tile_size != 0:
            print(f"⚠️ Pomijam {fname}, bo nie dzieli się równomiernie przez {tile_size}")
            continue

        labels = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f.readlines():
                    cls, cx, cy, w, h = map(float, line.strip().split())
                    labels.append((cls, cx, cy, w, h))

        tile_id = 0
        for y in range(0, height, tile_size):
            for x in range(0, width, tile_size):
                cropped = img.crop((x, y, x + tile_size, y + tile_size)).convert("RGB")
                new_img_name = f"{name}_{tile_id}.tif"
                output_path = os.path.join(output_img_dir, new_img_name)
                cropped.save(output_path, format="TIFF")

                new_labels = []
                for cls, cx, cy, w, h in labels:
                    xmin, ymin, xmax, ymax = convert_bbox((cx, cy, w, h), width, height)

                    if xmin >= x and xmax <= x + tile_size and ymin >= y and ymax <= y + tile_size:
                        new_cx = ((xmin + xmax) / 2 - x) / tile_size
                        new_cy = ((ymin + ymax) / 2 - y) / tile_size
                        new_w = (xmax - xmin) / tile_size
                        new_h = (ymax - ymin) / tile_size
                        new_labels.append(f"{int(cls)} {new_cx:.6f} {new_cy:.6f} {new_w:.6f} {new_h:.6f}")

                with open(os.path.join(output_lbl_dir, f"{name}_{tile_id}.txt"), "w") as f:
                    for line in new_labels:
                        f.write(line + "\n")

                print(f"🖼️ Zapisano: {output_path}")
                tile_id += 1


if __name__ == "__main__":
    print(f"🚀 Start — dzielenie na tile {tile_size}x{tile_size}")
    crop_images_and_labels()
    print("✅ Gotowe!")


🚀 Start — dzielenie na tile 1024x1024
🖼️ Zapisano: C:/Users/stszy/jupyterNoteBookNotes/cellDetector/model/LearnSet145\images_sliced\train\tile_12544_51968_0.tif
🖼️ Zapisano: C:/Users/stszy/jupyterNoteBookNotes/cellDetector/model/LearnSet145\images_sliced\train\tile_12544_51968_1.tif
🖼️ Zapisano: C:/Users/stszy/jupyterNoteBookNotes/cellDetector/model/LearnSet145\images_sliced\train\tile_12544_51968_2.tif
🖼️ Zapisano: C:/Users/stszy/jupyterNoteBookNotes/cellDetector/model/LearnSet145\images_sliced\train\tile_12544_51968_3.tif
🖼️ Zapisano: C:/Users/stszy/jupyterNoteBookNotes/cellDetector/model/LearnSet145\images_sliced\train\tile_12544_53760_0.tif
🖼️ Zapisano: C:/Users/stszy/jupyterNoteBookNotes/cellDetector/model/LearnSet145\images_sliced\train\tile_12544_53760_1.tif
🖼️ Zapisano: C:/Users/stszy/jupyterNoteBookNotes/cellDetector/model/LearnSet145\images_sliced\train\tile_12544_53760_2.tif
🖼️ Zapisano: C:/Users/stszy/jupyterNoteBookNotes/cellDetector/model/LearnSet145\images_sliced\train\t